### 필요한 라이브러리 호출

In [1]:
from langchain.document_loaders import PyPDFLoader
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_experimental.text_splitter import SemanticChunker
from langchain_ollama import ChatOllama
from langchain_chroma import Chroma
from langchain.retrievers.multi_query import MultiQueryRetriever
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.chains import create_history_aware_retriever, create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

### 문서 로드/분할 및 벡터 임베딩

In [2]:
# Chroma DB에서 문서 삭제
Chroma().delete_collection()

In [3]:
# PDF 파일 로드
loader = PyPDFLoader("data/대한민국헌법(헌법)(제00010호)(19880225).pdf")
pages = loader.load()

# 문서 분할
embeddings = HuggingFaceEmbeddings(model_name="snunlp/KR-SBERT-V40K-klueNLI-augSTS")
text_splitter = SemanticChunker(embeddings)
docs = text_splitter.split_documents(pages)

# LLM 선언
llm = ChatOllama(
    model = "basic_kanana",
    temperature = 0.7
)

# ChromaDB 선언
vectorstore = Chroma.from_documents(docs, embeddings)

# Retriever 선언
retriever_from_llm = MultiQueryRetriever.from_llm(
    retriever = vectorstore.as_retriever(
        search_type = "mmr",
        search_kwargs = {"lambda_mult": 0.7, "fetch_k": 10, "k": 3}
    ),
    llm = llm
)

/opt/anaconda3/envs/RAG_CODE/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### 채팅 히스토리와 사용자 질문 통합

In [4]:
# 채팅 히스토리-사용자 질문 통합 작업 지시 프롬프트
contextualize_q_system_prompt = """
대화 기록과 최신 사용자 질문이 주어졌을 때, 해당 질문이 대화 기록의 맥락을 참조하고 있을 수 있습니다. \
대화 기록 없이도 이해할 수 있도록 질문을 독립적인 형태로 재구성하세요. \
필요하지 않다면 원래 질문을 그대로 반환하세요. \
절대로 질문에 답하지 마세요.
"""

# 프롬프트 템플릿 생성
contextualize_q_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", contextualize_q_system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)

# 대화 기록을 고려한 retriever 생성
history_aware_retriever = create_history_aware_retriever(llm, retriever_from_llm, contextualize_q_prompt)

### RAG 체인 구축

In [5]:
# 컨텍스트와 사용자 질문을 함께 다룰 수 있도록 만들어주는 시스템 프롬프트
qa_system_prompt = """
당신은 질의응답 작업을 위한 어시스턴트입니다. \
다음에 제공된 문맥 정보를 활용하여 질문에 답하십시오. \
답을 모를 경우, 모른다고만 답하십시오. \
최대 세 문장으로 간결하게 답변하십시오. \
존댓말로 대답하십시오. \
{context}
"""

# 프롬프트 템플릿 생할
qa_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", qa_system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)

# Document 객체들을 하나의 텍스트로 묶기 및 LLM에 전달하는 역할
question_answer_chain = create_stuff_documents_chain(llm, qa_prompt)

# 최종 체인 생성
rag_chain = create_retrieval_chain(history_aware_retriever, question_answer_chain)

### RAG 체인 사용 방법 및 해팅 히스토리 기록

In [ ]:
# from langchain_core.messages import HumanMessage

# # 채팅 히스토리를 적재하기 위한 리스트
# chat_history = []

# question = "대통령의 임기는 몇년이야?"
# # 첫번째 질문에 답변하기 위한 rag_chain 실행
# ai_msg_1 = rag_chain.invoke({"input": question, "chat_history": chat_history})
# # 첫번째 질문과 답변을 채팅 히스토리로 저장
# chat_history.extend([HumanMessage(content=question), ai_msg_1["answer"]])

# second_question = "국회의원은?"
# # 두번째 질문 입력 시에는 첫번째 질문-답변이 저장된 chat_history가 삽입됨
# ai_msg_2 = rag_chain.invoke({"input": second_question, "chat_history": chat_history})

# print(ai_msg_2["answer"])

### 채팅 세션별 기록 자동 저장 RAG 체인 구축
- 채팅의 세션별로 서로 다른 채팅 히스토리를 저장, 개별의 대화방 생성

In [6]:
# 채팅 세션별 기록 저장 위한 Dictionary 선언
store = {}

# 주어진 session_id 값에 매칭되는 채팅 히스토리 가져오는 함수 선언
def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]

# RunnableWithMessageHistory 모듈로 rag_chain에 채팅 기록 세션별로 자동 저장 기능 추가
conversational_rag_chain = RunnableWithMessageHistory(
    rag_chain,
    get_session_history,
    input_messages_key = "input",
    history_messages_key = "chat_history",
    output_messages_key = "answer",
)

In [7]:
conversational_rag_chain.invoke(
    {"input": "대통령의 임기는 몇년이야?"},
    config={"configurable": {"session_id": "240510101"}},
)["answer"]

'대통령의 임기는 5년입니다.'

In [8]:
conversational_rag_chain.invoke(
    {"input": "국회의원은?"},
    config={"configurable": {"session_id": "240510101"}},
)["answer"]

'국회의원의 임기는 4년입니다. '